# day 1

In [ ]:
data = parse.(Int, readlines("./day1/input"))
[(i, j, i*j) for i in data for j in data if i+j == 2020]

In [ ]:
[(i, j, k, i*j*k) for i in data for j in data for k in data if i+j+k == 2020]

# day 2

In [ ]:
input = split.(readlines("./day2/input"))
data = [vcat(parse.(Int, split(i[1], "-")), [i[2][1], i[3]]) for i in input]

In [ ]:
sum([ i[1] <= count(c -> (c == i[3]), i[4]) <= i[2] for i in data])

In [ ]:
sum([ xor(i[4][i[1]] == i[3], i[4][i[2]] == i[3]) for i in data])

# day 3

In [ ]:
input = readlines("./day3/input")
data = input

function trees_hit(data::Array, right::Int=3, down::Int=1)::Int
    row = 1
    col = 1
    maxcols = length(data[1])
    maxrows = length(data)-1
    trees = 0

    for i = 1:maxrows/down
        row = row + down
        col = col + right
        if col > maxcols
            col = col - maxcols
        end
        if data[row][col] == '#'
            trees += 1
        end
    end
    return trees
end

res = 1
for (right, down) = [(1,1), (3,1), (5,1), (7,1), (1,2)]
    th = trees_hit(input, right, down)
    println(th)
    res *= th
end

res

# day 4

In [ ]:
input = replace.(split(read("./day4/input", String), "\n\n"), r"\n" => s" ")

data = [split.(pp) for pp in input]
data = [Dict(split.(pp, ":")) for pp in data]

valid_fields = ["hcl", "ecl", "pid", "cid", "iyr", "eyr", "hgt", "byr"]
sum([issubset(setdiff(valid_fields, keys(pp)), ["cid"]) for pp in data])

# day 5

In [ ]:
input = readlines("./day5/input");

In [ ]:
function parse_seat(seat_str, high='B')
    len = length(seat_str)
    coeffs = [c == high ? 1 : 0 for c in seat_str]
    return sum([c * 2^(len-pow) for (pow, c) in enumerate(coeffs)])
end

function seat_id(seat_str)
    return 8 * parse_seat(seat_str[1:7]) + parse_seat(seat_str[8:10], 'R')
    
end

seat_ids = seat_id.(input)
maximum(seat_ids)

In [ ]:
function find_seat(seat_ids)
    i1 = 0
    for i in sort(seat_ids)
        if i == i1+2
            return i-1
        end
        i1 = i
    end
end
    
find_seat(seat_ids)

# day 6

In [ ]:
using Test

function read_input(path)
    # chomp trailing \n, otherwise interpreted as empty set
    input = chomp(read(path, String))
    data = split(input, "\n\n")
    return data
end

function any_persons_answers(data)
    group_answers = delete!.(Set.(data), '\n')
    return group_answers
end

function all_persons_answers(data)
    person_answers = split.(data, '\n')
    every_person_answers = [ intersect(Set.(i)...) for i in person_answers ]
    return every_person_answers
end

sum_count(answers) = sum(length.(answers))

@test sum_count(any_persons_answers(read_input("./day6/test"))) == 11
@test sum_count(all_persons_answers(read_input("./day6/test"))) == 6

data = read_input("./day6/input")


print("$(sum_count(any_persons_answers(data))), $(sum_count(all_persons_answers(data)))")

# day 7

In [ ]:
function parse_input(path)
    data = split.(chop.(replace.(readlines(path), r" bag[s]*" => "")), " contain ")
    data = Dict(map(i -> [i[1], Dict{Any,Any}([parse_arguments(s) for s in split(i[2], ", ") if s[1:2] != "no"])], data))
    return data
end

function parse_arguments(str)
    return [str[3:end], parse(Int, str[1])]
end

function build_tree(key::String, rules)
    tree = copy(rules[key])
    for (k, v) in tree
        tree[k] = build_tree(convert(String, k), rules)
    end
    return tree
end

function build_tree(dict::Dict, rules)
    tree = copy(dict)
#     tree = dict
    for (k, v) in tree
        tree[k] = build_tree(rules[k], rules)
    end
    return tree
end

function find_in_tree(tree, key::String)
    for (k, v) in tree
        if k == key
            return true
        else
            found = find_in_tree(v, key)
            if found
                return found
            end
        end
    end
    return false
end

function count_bags(tree, rules, key)
    count = 0
    for (k, subtree) in tree[key]
        if length(keys(subtree)) == 0
            count += rules[key][k]
        else
            count += rules[key][k] * (count_bags(tree[key], rules, k) + 1) # +1 for counting the bag itself
        end
    end
    return count
end


data = parse_input("./day7/input")

# @time tree = Dict(k => build_tree(convert(String, k), data) for (k, v) in data)
@time tree = build_tree(data, data)
@show sum([ find_in_tree(v, "shiny gold") for (k, v) in tree
            if (k != "shiny gold") & (find_in_tree(v, "shiny gold") == 1) ])
@show count_bags(tree, data, "shiny gold")

# data
# tree


# day 8

In [ ]:
mutable struct ProgramInstance
    code::Array{Any, 2}
    acc::Int
    idx::Int
    idx_history::Array{Int, 1}
    ProgramInstance(code, acc, idx) = new(code, acc, idx, [idx])
end

function Base.show(io::IO, prog::ProgramInstance)
   print("idx=$(prog.idx), acc=$(prog.acc)") 
end


function read_input(path)
    input = readlines(path)
    code = split.(input)
    code = Array{Any, 2}(hcat(split.(input)...))
    code[2,:] = parse.(Int, code[2,:])
    return permutedims(code)
end

function parse_code(prog::ProgramInstance)
    cmd, val = prog.code[prog.idx, :]
    if cmd == "jmp"
        prog.idx += val
    elseif cmd == "acc"
        prog.acc += val
        prog.idx += 1
    elseif cmd == "nop"
        prog.idx += 1
    end
#     return prog
end

function debug(prog)
    if prog.idx in Set(prog.idx_history)
#             print("acc=$acc, $(code[idx,1])=$(code[idx,2]), $idx, $idx_history")
        println("Infinite Loop. acc=$(prog.acc), idx=$(prog.idx)")
        return 1
    elseif prog.idx <= 0
        println("Negative index. acc=$(prog.acc), idx=$(prog.idx)")
        return 1
    elseif prog.idx > size(code, 1)
        println("Success! acc=$(prog.acc), idx=$(prog.idx)")
        return 0
    end
    return -1
end

function run(code)
    prog = ProgramInstance(code, 0, 1)
    
    exit_code = -1
    while true
        parse_code(prog)
        exit_code = debug(prog)
        
        if exit_code >= 0
            break
        end

        push!(prog.idx_history, prog.idx)
    end
    
    return exit_code, prog.acc
end

function modify_code(code, idx, to="jmp")
    code_copy = copy(code)
    code_copy[idx, 1] = to
    return code_copy
end

code = read_input("./day8/test")
run(code)

In [ ]:
code = read_input("./day8/input")

exit_code = -1
acc = 0
for idx in 1:size(code,1)
    if code[idx, 1] == "nop"
        exit_code, acc = run(modify_code(code, idx, "jmp"))
    elseif code[idx, 1] == "jmp"
        exit_code, acc = run(modify_code(code, idx, "nop"))
    end
    if exit_code == 0
        println(acc)
        break
    end
end

# day 9

In [ ]:
function validate(data, idx, range)
    data_range = data[idx-range:idx-1]
    resid = data[idx] .- data_range
    is_not_used_twice = resid .!= data_range
    is_in_range = [ r in Set(data_range) for r in resid ] # afaik no broadcasting for 'in'
    return |((is_not_used_twice .& is_in_range)...)
end

function find_invalid(data, range)
    for idx in (range+1):length(data)
        if validate(input, idx, range) == false
            invalid = input[idx]
            return idx, invalid
        end
    end
end

find_invalid(parse.(Int, readlines("./day9/input")), 25)

In [ ]:
function get_sum_range(data, idx, thresh)
    total = cumsum(data) .- sum(input[1:idx])
    sum_range = data[findall(0 .<= total .<= thresh)]
    return sum_range, sum(sum_range)
end

input = parse.(Int, readlines("./day9/input"))
idx, thresh = find_invalid(input, 25)


for i in 1:idx-1
    sum_range, sum_sum_range = get_sum_range(input, i, thresh)
    if sum_sum_range == thresh
        return sum_range, maximum(sum_range) + minimum(sum_range)
    end
end

# day 10

In [ ]:
using StatsBase

input = parse.(Int, readlines("./day10/input"))
append!(input, [0, maximum(input)+3])
data = sort(input)
data_diff = data[2:end] - data[1:end-1]
occurences = countmap(data_diff)
occurences[1] * occurences[3]

In [ ]:
using Combinatorics
using IterTools

function perm_length(len)
    """only works for step sizes up to 3"""
    choosearr = vcat(Int.(ones(len)), Int.(2 * ones(Int((len-mod(len,2))/2))), Int.(3 * ones(Int((len-mod(len,3))/3))))

    subs = unique([i for i in subsets(choosearr) if sum(i) == len])
    return size(unique([i for perm in permutations.(subs) for i in perm]), 1)
end

occ, occ_len = rle(data_diff) # run-length-encoding

*([perm_length(occ_len[i]) for (i, o) in enumerate(occ) if o == 1]...)

# day 12

In [ ]:
mutable struct Boat
    rot::Int
    x::Int
    y::Int
end

function parse_input(s)
    rx = r"([A-Z])([0-9]+)"
    c = match(rx, s).captures
    return c[1], parse(Int, c[2])
end

function navigate(b::Boat, dir, val)
    if dir == "E"
        b.x += val
    elseif dir == "W"
        b.x -= val
    elseif dir == "N"
        b.y += val
    elseif dir == "S"
        b.y -= val
    elseif dir == "L"
        b.rot += val
    elseif dir == "R"
        b.rot -= val
    elseif dir == "F"
        b.x += val * round(Int, cos(pi/180*b.rot))
        b.y += val * round(Int, sin(pi/180*b.rot))
    end
    return b
end

function norm_L1(vec)
    return sum(abs.(vec))
end

function run(path)
    input = readlines(path)
    data = parse_input.(input)
    b = Boat(0, 0, 0)
    for i in data
        navigate(b, i...)
    end
    return norm_L1((b.x, b.y))
end

run("./day12/input")

In [ ]:
mutable struct WaypointBoat
    boat_x::Int
    boat_y::Int
    wp_x::Int
    wp_y::Int
end

function rotate(rot, x, y)
    rot_matrix = round.(Int, [ cos(pi/180*rot) -sin(pi/180*rot) ; sin(pi/180*rot) cos(pi/180*rot)])
    return rot_matrix * [x, y]
end

function navigate_waypoint(b::WaypointBoat, dir, val)
    if dir == "E"
        b.wp_x += val
    elseif dir == "W"
        b.wp_x -= val
    elseif dir == "N"
        b.wp_y += val
    elseif dir == "S"
        b.wp_y -= val
    elseif dir == "L"
        b.wp_x, b.wp_y = rotate(val, b.wp_x, b.wp_y)
    elseif dir == "R"
        b.wp_x, b.wp_y = rotate(-val, b.wp_x, b.wp_y)
    elseif dir == "F"
        b.boat_x += val * b.wp_x
        b.boat_y += val * b.wp_y
    end
    return b
end

function run_waypoint(path, init_x, init_y)
    input = readlines(path)
    data = parse_input.(input)
    b = WaypointBoat(0, 0, init_x, init_y)
    for i in data
        navigate_waypoint(b, i...)
    end
    return norm_L1((b.boat_x, b.boat_y))
end

run_waypoint("./day12/input", 10, 1)

# day 13

In [ ]:
arrival_time, schedules = readlines("./day13/input")
arrival_time = parse(Int, arrival_time)

ids = [parse(Int, r.match) for r in eachmatch(r"([0-9]+)", schedules)]
arrivals = [(arrival_time % i == 0 ? arrival_time % i : i - arrival_time % i, i) for i in ids]
prod(minimum(arrivals))

In [ ]:
function find_least_common_offset_multiple(delays; t0=0)
    o, dt = partialsort(delays, 1, by=i->i[2], rev=true)
    t = div(t0, dt) * dt
    @show t
    print_t = 10^10
    print_i = div(t, print_t) + 1
    while true
        t += dt
        if sum([(t+d-o) % id for (d, id) in delays]) == 0
            return t-o
        end
        if div(t, print_t) >= print_i
            println("current t = $t")
            print_i += 1
        end
    end
end


arrival_time, schedules = readlines("./day13/input")
schedules = "23,41"

delays = [ [t-1, parse(Int, id)] for (t, id) in enumerate(split(schedules, ",")) if id != "x"]

find_least_common_offset_multiple(delays, t0=0)

# day 14

In [ ]:
function apply_bitmask(int, mask)
    b_in = bitstring(UInt64(int))[end-35:end]
    b_out = *([b == 'X' ? b_in[i] : b for (i, b) in enumerate(mask)]...)
    return parse(Int, b_out, base=2)
end


input = readlines("./day14/input")

mem = zeros(Int, 65200)

for s in input
    if s[1:4] == "mask"
        eval(Meta.parse(replace(s, r"([01X]+)" => s"\"\1\"")))
    else
        eval(Meta.parse(replace(s, r" ([0-9]+)" => s" apply_bitmask(\1, mask)")))
    end
end

sum(mem)

# day 15

In [ ]:
function play(starting_numbers, duration)
    hist = copy(starting_numbers)
    for i in 1:duration - length(hist)
        if hist[end] in hist[1:end-1]
            push!(hist, length(hist) - findlast(hist[1:end-1] .== hist[end]))
        else
            push!(hist, 0)
        end
    end
    return hist
end

starting_numbers = [1,12,0,20,8,16]
starting_numbers = [0,3,6]


@time play(starting_numbers, 2020)

# play(starting_numbers, 30000000)[end]

In [ ]:
function play3(starting_numbers::Array{Int,1}, N::Int)
    lastpos = Dict{Int,Int}()
    for (i, v) in enumerate(starting_numbers[1:end-1])
        lastpos[v] = i
    end
    last = starting_numbers[end]
    for i = length(starting_numbers) : N-1
        if haskey(lastpos, last)
            tmp = last
            last = i - lastpos[last]
            lastpos[tmp] = i
        else
            lastpos[last] = i
            last = 0
        end
    end
    return last
end

starting_numbers = [1,12,0,20,8,16]

@time play3(starting_numbers, 30000000)

In [ ]:
using OffsetArrays


function play4(starting_numbers::Array{Int,1}, N::Int)
    lastpos = OffsetArray(zeros(Int32, N), 0:N-1)
    for (i, v) in enumerate(starting_numbers[1:end-1])
        lastpos[v] = i
    end
    last = starting_numbers[end]
    for i = length(starting_numbers) : N-1
        if lastpos[last] == 0
            lastpos[last] = i
            last = 0    
        else
            tmp = last
            last = i - lastpos[last]
            lastpos[tmp] = i
        end
    end
    return last
end

starting_numbers = [1,12,0,20,8,16]

@time play4(starting_numbers, 30000000)

# day 16

In [ ]:
function parse_invalids(s)
    bounds = parse.(Int, match(r"([0-9]+) or ([0-9]+)", s).captures)
    return bounds[1]+1 : bounds[2]-1
end

function parse_valids(s)
    bounds = parse.(Int, match(r"([0-9]+)-([0-9]+) or ([0-9]+)-([0-9]+)", s).captures)
    return bounds[1]:bounds[2], bounds[3]:bounds[4]
end

rules, my_ticket, tickets = split(read("./day16/input", String), "\n\n")
rules = Dict(split.(split(rules, "\n"), ": "))
my_ticket = map(s -> parse(Int, s), split.(split(chomp(my_ticket), "\n")[2], ","))
tickets = map(a -> parse.(Int, a), split.(split(chomp(tickets), "\n")[2:end], ","))

valid_rules = Dict(r => vcat(parse_valids(s)...) for (r, s) in rules)
valid_ranges = Set(vcat([r for (k, r) in valid_rules]...))

error_rate = 0
valid_tickets = []
for ticket in tickets
    invalid_index = findfirst(map(n -> n in valid_ranges, ticket) .== 0)
    if isnothing(invalid_index) == false
        error_rate += ticket[invalid_index]
    else
        push!(valid_tickets, ticket)
    end
end

@show error_rate



tickets_mat = hcat(valid_tickets...)'

function find_valid_mapping(possible_mappings)
    pm = copy(possible_mappings)
    valid_mapping = Dict()
    iter_order = Dict(length(v) => k for (k, v) in pm)
    for i in 1:length(iter_order)
        m = pop!(pm, iter_order[i])[1]
        valid_mapping[m] = iter_order[i]
        pm = Dict(k => filter(i -> i!=m, v) for (k, v) in pm)
    end
    
    return valid_mapping
end

possible_mappings = Dict()
valid_mapping = Dict()

for (k, v) in valid_rules
    valid_pos = [ all(in.(tickets_mat[:,i], (valid_rules[k],))) for i in 1:size(tickets_mat)[2] ]
    possible_mappings[k] = findall(valid_pos)
end

valid_mapping = find_valid_mapping(possible_mappings)

score = 1
for (pos, field) in enumerate(my_ticket)
    if haskey(valid_mapping, pos) 
        if (split(valid_mapping[pos], " ")[1] == "departure")
        score *= field
        end
    end
end
@show score

# day 17

In [16]:
using OffsetArrays


function get_init(path)
    input = readlines(path)
    init = hcat([map(i -> i == '#' ? 1 : 0, collect(s)) for s in input]...)
    return init
end

function initialize(init::AbstractArray{Int}, dim::Int, N::Int)
    dims = fill(N, dim)
    A = OffsetArray(falses(dims...), -cld.(dims, 2)...)
    init = reshape(init, Val(dim))
    init_centered = OffsetArray(init, -cld.(collect(size(init)), 2)...)
    init_axes = axes(init_centered)
        
    A_init = view(A, init_axes...)
    A_init[:] = init_centered
    
    inds = CartesianIndices(init_centered)
    i_min = first(inds)
    i_max = last(inds)
    i1 = oneunit(i_min)

    return @view A[i_min-i1 : i_max+i1]
end

function apply_rules(el::Bool, agg::Int)
    if el == 1
        return agg in 3:4 ? 1 : 0
    else
        return agg == 3 ? 1 : 0
    end
end

function evolve!(A::AbstractArray{Bool})
    """works with arrays and views"""
    A_new = similar(A) # does not inherit custom indices
    inds = CartesianIndices(A)
    i_min = first(inds)
    i_max = last(inds)
    i1 = oneunit(i_min)
    for i in inds
        s = zero(eltype(A))
        for j in max(i-i1, i_min) : min(i+i1, i_max)
            s += A[j]
        end
        A_new[i] = apply_rules(A[i], s)
    end
    A[:] = A_new
    return A
end

function get_active(A::AbstractArray{Bool})
    """returns active indices of array, or parent array of view"""
    if typeof(A) <: SubArray
        inds = parentindices(A)[1] # not sure why first element
        A = parent(A)
    else
        inds = CartesianIndices(A)
    end
    active_inds = []
    for i in inds
        if A[i] == 1
            push!(active_inds, i)
        end
    end
    return active_inds
end

function adjust_size(A::AbstractArray{Bool})
    active_inds = get_active(A)
        
    if typeof(A) <: SubArray
        A = parent(A)
    end
    
    i_min = minimum(active_inds)
    i_max = maximum(active_inds)
    i1 = oneunit(i_min)    

    return @view A[i_min-i1:i_max+i1]
end


function run(init::AbstractArray{Int}, dim::Int, N::Int, t::Int)
    A = initialize(init, dim, N)
    active_t = Vector{Array{Any}}(undef, t)
    for i in 1:t
        A = adjust_size(A)
        evolve!(A)
        active_t[i] = get_active(A)
    end
    return A, active_t
end


init = get_init("./day17/input")
@time A, active_t = run(init, 2, 50, 50)
length(get_active(A))


  1.319280 seconds (4.00 M allocations: 198.792 MiB, 8.97% gc time)


86

In [ ]:
using Plots
plotly()
heatmap(z=[1 2 3;3 2 1;2 3 1])
# plot(length.(active_t))

In [ ]:
function initialize(init::AbstractArray{Int}, dim::Int, N::Int)
    dims = fill(N, dim)
    A = OffsetArray(falses(dims...), -cld.(dims, 2)...)
    init = reshape(init, Val(dim))
    init_centered = OffsetArray(init, -cld.(collect(size(init)), 2)...)
    init_axes = axes(init_centered)
        
    A_init = view(A, init_axes...)
    A_init[:] = init_centered
    
    inds = CartesianIndices(init_centered)
    i_min = first(inds)
    i_max = last(inds)
    i1 = oneunit(i_min)

    return @view A[i_min-i1 : i_max+i1]
end

function evolve!(A_view::AbstractArray{Bool})
    """adjust size in same cycle
    still buggy..."""
    inds = parentindices(A_view)[1] # not sure why first element
    A = parent(A_view)

    i_min = first(inds)
    i_max = last(inds)
    i1 = oneunit(i_min)
    
    A_new = similar(A) # does not inherit custom indices
    fill!(A_new, zero(eltype(A)))
    
    active_inds = []
    for i in i_min-i1 : i_max+i1
        s = zero(eltype(A))
        for j in i-i1 : i+i1
            s += A[j]
        end
        Ai = A[i]
        A_new[i] = apply_rules(Ai, s)
        
        if Ai == 1
            push!(active_inds, i)
        end
    end
    A[:] = A_new
    
    i_min = minimum(active_inds) - i1
    i_max = maximum(active_inds) + i1

    return view(A, i_min:i_max), active_inds

end

function run(init::AbstractArray{Int}, dim::Int, N::Int, t::Int)
    A = initialize(init, dim, N)
    active_inds = []
    for i in 1:t
        A, active_inds = evolve!(A)
    end
    return A, active_inds
end

init = get_init("./day17/test")

@time A, ai = run(init, 3, 30, 6)
length(ai)
A


# day 18

In [13]:
import Base.∘ # Base.operator_precedence(:∘) same as :*

∘(a::Integer, b::Integer) = a + b
⊕(a::Integer, b::Integer) = a * b

input = readlines("./day18/input")
data = replace.(input, "+" => "∘")
@show sum(eval.(Meta.parse.(data)))

data = replace.(data, "*" => "⊕")
@show sum(eval.(Meta.parse.(data)))

sum(eval.(Meta.parse.(data))) = 2743012121210
sum(eval.(Meta.parse.(data))) = 65658760783597


65658760783597

# day 19

In [2]:
function parse_input(path)
    rules, data = split(read(path, String), "\n\n")
    data = split(chomp(data), "\n")

    rules = split(rules, "\n")
    rules = Dict(split.(rules, ": "))
#     rules = Dict(k => convert(Array{Array{Any,1},1}, split.(split(v, " | "), " ")) for (k,v) in rules)
    
    function parse_arr(s) 
        p = string("[" * replace(replace(s, r"(\d+ \d+|\d+)" => s"[\1]"), " | " => " , ") * "]") |> Meta.parse |> eval
        if typeof(p[1]) == String
            p = p[1]
        else
            p = vec.(p)
        end
        return p
    end
    
    rules = Dict(parse(Int, k) => parse_arr(v) for (k, v) in rules)
    
    return rules, data
end

function replace_recursive(arr::Union{Array{Array{Int,1},1},String}, rules)
    tree = Array{Array{Any,1},1}(copy(arr))
    for (r1, ruleset) in enumerate(arr)
        for (r2, rule) in enumerate(ruleset)
            subrule = rules[rule]
            if typeof(subrule) != String
                tree[r1][r2] = replace_recursive(subrule, rules)
            else
                tree[r1][r2] = subrule
            end
        end 
    end
    return tree
end


function create_valids(tree)::Array{Any,1}
    choices = []
    substrs = []
    for choice in tree
        tmp = []
        for subtree in choice
            if typeof(subtree) != String
                push!(tmp, create_valids(subtree))
            else
                push!(tmp, [subtree])
            end
        end
        if length(tmp) == 2
            substrs = [s1*s2 for s1 in tmp[1] for s2 in tmp[2]]
        else
            substrs = [s1 for s1 in tmp[1]]
        end
        append!(choices, substrs)
    end
    
    return choices
end

rules, data = parse_input("./day19/input")

tree = replace_recursive(rules[0], rules)
valids = create_valids(tree)
@time valid_data = in.(data, (valids,))

sum(valid_data)

 47.828751 seconds (1.41 G allocations: 63.109 GiB, 8.82% gc time)


136

# day 20

In [104]:
using DataStructures

mutable struct Tile
    id::Int
    mat::BitArray{2}
    borders::BitArray{2}
    placed_borders::Vector{Int}
    Tile(id, mat) = new(id, mat, [mat[:,1] mat[1,:] mat[:,end] mat[end,:] mat[end:-1:1,1] mat[1,end:-1:1] mat[end:-1:1,end] mat[end,end:-1:1]])
end

function get_tiles(path)
    input = split.(split(chop(read(path, String), tail=2), "\n\n"), "\n")
    tiles = Dict{Int, Tile}()
    for t in input
        id = parse(Int, t[1][6:9])
        mat = BitArray(hcat([map(i -> i == '#' ? true : false, collect(s)) for s in t[2:end]]...))
        tiles[id] = Tile(id, mat)
    end
    return tiles
end

function get_border_tiles(tiles)
    corners = DefaultDict{Int, Vector{Int}}(Vector{Int})
    borders = DefaultDict{Int, Vector{Int}}(Vector{Int})
    
    for (id, tile) in tiles
        for (ci, c) in enumerate(eachcol(tile.borders))
            match = false
            for (_, tile2) in tiles
                if tile != tile2
                    for c2 in eachcol(tile2.borders)
                        if c == c2
                            match = true
                            break
                        end
                    end
                    if match == true
                        break
                    end
                end
            end
            if match == false
                if length(borders[id]) == 2
                    if !haskey(corners, id)
                        corners[id] = borders[id]
                    end
                    push!(corners[id], ci)
                else
                    push!(borders[id], ci)
                end
            end
        end
        if haskey(corners, id)
            delete!(borders, id)
        end
    end
    
    return borders, corners
end

tiles = get_tiles("./day20/input")

borders, corners = get_border_tiles(tiles)

@show corners |> keys |> prod

(corners |> keys) |> prod = 16937516456219


16937516456219

In [105]:
corners

DefaultDict{Int64,Array{Int64,1},DataType} with 4 entries:
  2347 => [1, 2, 5, 6]
  2281 => [1, 2, 5, 6]
  1811 => [3, 4, 7, 8]
  1747 => [1, 2, 5, 6]

In [106]:
pop!(corners, 2347)

4-element Array{Int64,1}:
 1
 2
 5
 6

In [ ]:
function place_border_tile(tiles, borders, corners, id_map)
    
end

function place_corner_tile(tiles)
    
end
    
    
len = Int(sqrt(length(tiles)))
id_map = Array{Int,2}(undef, (len, len))
    
id_map[1,1] = 2347
pop!(corners, 2347)
tiles[2347].placed_borders = [1,2,3,4]

In [102]:
len = Int(sqrt(length(tiles)))
id_map = Array{Int,2}(undef, (len, len))

init = Tuple(pop!(corners))
init_tile = tiles[init[1]]

for (id, b) in borders
    b_tile = tiles[id]
    for (ci, c) in enumerate(eachcol(b_tile.borders))
#         if !(ci in b)
            if init_tile.borders[:,3] == c
                @show id, ci, c
            end
#         end
    end
end
    

(id, ci, c) = (1601, 4, Bool[0, 0, 1, 0, 1, 1, 0, 0, 0, 1])


In [ ]:
# transformations
[1 2 3 4]
[2 3 4 1]
[3 4 1 2]
[4 1 2 3]

[5 1 7 4]
[1 6 3 8]

[2 1 4 3]
[8 7 6 5]

# day 22

In [ ]:
input = map(s -> s[11:end], split(chomp(read("./day22/input", String)), "\n\n"))
deck_p1, deck_p2 = split.(input, "\n")
deck_p1 = parse.(Int, deck_p1)
deck_p2 = parse.(Int, deck_p2)

In [ ]:
for r in eachrow([deck_p1 deck_p2])
    
end

In [ ]:
maximum.([deck_p1 deck_p2])

In [ ]:
deck_p1[deck_p1 - deck_p2 .> 0]

In [ ]:
function play_combat(deck_p1, deck_p2)
    while length(deck_p1) * length(deck_p2) > 0
        if deck_p1[1] > deck_p2[1]
            deck_p1 = append!(deck_p1[2:end], [deck_p1[1], deck_p2[1]])
            deck_p2 = deck_p2[2:end]
        elseif deck_p1[1] < deck_p2[1]
            deck_p2 = append!(deck_p2[2:end], [deck_p2[1], deck_p1[1]])
            deck_p1 = deck_p1[2:end]
        end
    end
    if length(deck_p1) > 0
        return deck_p1
    else
        return deck_p2
    end
end
    
function score(deck)
    return sum([pos*val for (pos, val) in enumerate(reverse(deck))])
end

score(play_combat(deck_p1, deck_p2))